In [5]:
import numpy as np
from scipy.stats import norm

# Newton-Ralphsons Method Applied To Finance

## Formula

$ \Large x_{i+1} = x_i - \dfrac{f(x_i)}{f'(x_i)} $

### Applications

1. Internal Rate of Return
2. Interest Rate Solving
3. Implied Volatility w/ Black Scholes

## IRR

$ \Large \begin{matrix} NPV = \sum_{t=0}^n \dfrac{x_t}{(1 + r)^t} \\ dNPV = -\dfrac{tx_t}{(1 + r)^{t+1}} \end{matrix} $

In [11]:
def INTERNALRATEOFRETURN():
    def IRR(x):
        
        def F(x, r):
            npv = 0
            for t, cf in enumerate(x):
                npv += cf/pow(1 + r, t)
            return npv
            
        def F1(x, r):
            npv = 0
            for t, cf in enumerate(x):
                npv += -t*cf/pow(1 + r, t + 1)
            return npv

        r0 = 0.01
        r1 = 0.99

        while True:
            r1 = r0 - F(x, r0)/F1(x, r0)
            if abs(r1 - r0) < 0.00001:
                break
            r0 = r1

        return r1

    cf = [-100, 20, 30, 40, 50, 60]
    irr = IRR(cf)

    npv = np.sum([j/pow(1 + irr, i) for i, j in enumerate(cf)])
    print(npv)


INTERNALRATEOFRETURN()

1.454480980100925e-11


In [13]:
def INTERESTRATE():

    def InterestRate(balance, x, t):
        def F(x, r, t):
            return (x/r)*(1 - pow(1 + r, -t))
    
        def F1(x, r, t):
            # AB' + BA'
            # A = x/r
            # B = 1 - (1+r)^-t
            # A1 = -x/r^2
            # B1 = t(1+r)^-t-1 -> t(1 +r)^-(t+1) -> t/(1 + r)^(t+1)
            A = x/r
            B = 1 - pow(1 + r, -t)
            A1 = -x/pow(r, 2)
            B1 = t/pow(1 + r, t+1)
            return A*B1 + B*A1

        r0, r1 = 0.01, 0.99
        while True:
            r1 = r0 - (F(x, r0, t) - balance)/F1(x, r0, t)
            if abs(r1 - r0) < 0.00001:
                break
            r0 = r1
        return r1

    balance = 500000
    payment = 43000
    time_payoff = 30

    interest_rate = InterestRate(balance, payment, time_payoff)

    value = (payment/interest_rate)*(1 - pow(1 + interest_rate, -time_payoff))
    print(value)

INTERESTRATE()

500000.00001928606


In [23]:
def BlackScholes():

    def DF(S, K, r, q, v, t):
        top = np.log(S/K) + (r - q + 0.5*pow(v, 2))*t
        bot = v*np.sqrt(t)
        d1 = top/bot
        d2 = d1 - bot
        return d1, d2

    def F(S, K, r, q, v, t):
        d1, d2 = DF(S, K, r, q, v, t)
        return S*np.exp(-q*t)*norm.cdf(d1) - K*np.exp(-r*t)*norm.cdf(d2)

    def F1(S, K, r, q, v, t):
        d1, d2 = DF(S, K, r, q, v, t)
        return S*np.exp(-q*t)*np.sqrt(t)*norm.pdf(d1)

    def ImpliedVolatility(opPrice, S, K, r, q, t):
        v0 = 0.1
        v1 = 0.55
        while True:
            v1 = v0 - (F(S, K, r, q, v0, t) - opPrice)/F1(S, K, r, q, v0, t)
            if abs(v1 - v0) < 0.00001:
                break
            v0 = v1
        return v1

    option_price = 2.33
    S = 100
    K = 105
    r = 0.05
    q = 0.01
    t = 0.5

    IV = ImpliedVolatility(option_price, S, K, r, q, t)

    d1, d2 = DF(S, K, r, q, IV, t)
    check_price = S*np.exp(-q*t)*norm.cdf(d1) - K*np.exp(-r*t)*norm.cdf(d2)
    print(check_price)

BlackScholes()

2.3300000000000836
